## 1. Import Libraries and Global Settings
Import required Python libraries, set random seed for reproducibility, and count dataset category information

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['ABSL_LOG_LEVEL'] = 'FATAL'
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
import random
import tensorflow as tf
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)


base_dir = '../data/garbage-dataset'
classes = os.listdir(base_dir)
train_dir = '../data/garbage-split/train'
test_dir = '../data/garbage-split/test'
val_dir = '../data/garbage-split/val'

class_names = sorted(classes)
num_classes = len(class_names)
print("Classes:", class_names)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Classes: ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']
Num GPUs Available: 1


## 2. Data Augmentation
Apply augmentation to training data and rescaling to validation data using `ImageDataGenerator`

In [2]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

def pytorch_normalize(img):
    img = img / 255.0
    return (img - mean) / std

train_datagen = ImageDataGenerator(
    preprocessing_function=pytorch_normalize,
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.1,
    zoom_range=[0.95, 1.05],
    brightness_range=[0.85, 1.15],
    horizontal_flip=True,
    channel_shift_range=0.02,
    fill_mode='reflect'
)

val_test_datagen = ImageDataGenerator(
    preprocessing_function=pytorch_normalize,
)

## 3. Data Generators
Load images from folders using `flow_from_directory` for training and validation sets

In [3]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=True,
    seed=SEED
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir, 
    target_size=(224, 224), 
    batch_size=16, 
    class_mode='categorical',
    shuffle=False
)

train_steps = int(np.ceil(train_generator.samples / 16))
val_steps = int(np.ceil(val_generator.samples / 16))

print(f"Number of training samples: {train_generator.samples}, Steps per epoch: {train_steps}")
print(f"Number of validation samples: {val_generator.samples}, Validation steps: {val_steps}")

Found 15806 images belonging to 10 classes.
Found 2963 images belonging to 10 classes.
Found 993 images belonging to 10 classes.
Number of training samples: 15806, Steps per epoch: 988
Number of validation samples: 2963, Validation steps: 186


## 4. Compute Class Weights
Handle class imbalance by assigning weights to different classes

In [4]:
y_train = train_generator.classes
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))
print('Class weights:', class_weight_dict)

Class weights: {0: 2.0935099337748344, 1: 1.9831869510664994, 2: 1.0826027397260274, 3: 0.3709457873738559, 4: 0.6456699346405229, 5: 1.9370098039215686, 6: 1.1760416666666667, 7: 0.9959672337744171, 8: 0.9997469955724225, 9: 2.0879788639365917}


## 5. Adjust Specific Class Weights
Increase weights for metal and trash classes to improve their recognition accuracy

In [5]:
class_weight_dict[1] *= 2.0  # biological
class_weight_dict[4] *= 1.7  # glass
class_weight_dict[5] *= 1.5 # metal
class_weight_dict[7] *= 1.2 # plastic
class_weight_dict[8] *= 1.5  # shoes
class_weight_dict[9] *= 2.5  # trash
print('Class weights:', class_weight_dict)

Class weights: {0: 2.0935099337748344, 1: 3.966373902132999, 2: 1.0826027397260274, 3: 0.3709457873738559, 4: 1.0976388888888888, 5: 2.9055147058823527, 6: 1.1760416666666667, 7: 1.1951606805293005, 8: 1.4996204933586337, 9: 5.219947159841479}


## 6. Build CNN Model

Create a CNN with Conv-BatchNorm-Pool blocks and global average pooling for classification.


In [6]:
model = Sequential([
    Input(shape=(224, 224, 3)),

    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(num_classes, activation='softmax')
])

## 7. Compile Model

Compile the model with Adam optimizer and categorical cross-entropy loss.


In [7]:
model.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## 8. Train Model

Train the model with early stopping, checkpointing, and learning rate scheduling.


In [8]:
checkpoint = ModelCheckpoint('saved_models/best_custom_cnn.keras',monitor='val_accuracy', mode='max', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, min_delta=0.001, verbose=1)
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.4, patience=4, min_lr=1e-6, verbose=1)

history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    validation_data=val_generator,
    validation_steps=val_steps,
    epochs=100,
    callbacks=[early_stop, lr_reduce, checkpoint],
    class_weight=class_weight_dict,
    verbose=1
)

Epoch 1/100


I0000 00:00:1753857784.173143    1248 service.cc:145] XLA service 0x7d6db001e860 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753857784.173235    1248 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti with Max-Q Design, Compute Capability 7.5


  2/988 ━━━━━━━━━━━━━━━━━━━━ 1:22 83ms/step - accuracy: 0.0781 - loss: 3.8893  

I0000 00:00:1753857806.701632    1248 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


988/988 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step - accuracy: 0.2687 - loss: 2.7754
Epoch 1: val_accuracy improved from -inf to 0.36011, saving model to saved_models/best_custom_cnn.keras
988/988 ━━━━━━━━━━━━━━━━━━━━ 241s 217ms/step - accuracy: 0.2687 - loss: 2.7753 - val_accuracy: 0.3601 - val_loss: 1.8650 - learning_rate: 5.0000e-04
Epoch 2/100
988/988 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.3724 - loss: 2.3556
Epoch 2: val_accuracy improved from 0.36011 to 0.42288, saving model to saved_models/best_custom_cnn.keras
988/988 ━━━━━━━━━━━━━━━━━━━━ 181s 183ms/step - accuracy: 0.3724 - loss: 2.3556 - val_accuracy: 0.4229 - val_loss: 1.6787 - learning_rate: 5.0000e-04
Epoch 3/100
988/988 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.4284 - loss: 2.1320
Epoch 3: val_accuracy did not improve from 0.42288
988/988 ━━━━━━━━━━━━━━━━━━━━ 176s 178ms/step - accuracy: 0.4284 - loss: 2.1321 - val_accuracy: 0.4225 - val_loss: 1.7378 - learning_rate: 5.0000e-04
Epoch 4/100
988/988 ━━━━━━━━━━━━━━━━

## 9. Evaluate and Save

Evaluate model on validation set and save final model.




In [9]:
loss, acc = model.evaluate(val_generator)
print(f'Validation Accuracy: {acc:.2f}')

model.save('saved_models/custom_cnn_final.keras')
print("Model has been saved as custom_cnn_final.keras")

186/186 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - accuracy: 0.8801 - loss: 0.4603
Validation Accuracy: 0.87
Model has been saved as custom_cnn_final.keras


## 11. Classification Report
Generate detailed classification report with precision, recall, and F1-score for each class

In [10]:
from sklearn.metrics import classification_report
test_generator.reset()
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

print(classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys())))

63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step
              precision    recall  f1-score   support

     battery       0.91      0.90      0.91        48
  biological       0.98      0.80      0.88        50
   cardboard       0.86      0.80      0.83        92
     clothes       0.96      0.92      0.94       267
       glass       0.92      0.90      0.91       154
       metal       0.77      0.84      0.80        51
       paper       0.75      0.87      0.81        84
     plastic       0.88      0.85      0.86       100
       shoes       0.78      0.82      0.80        99
       trash       0.78      0.94      0.85        48

    accuracy                           0.87       993
   macro avg       0.86      0.86      0.86       993
weighted avg       0.88      0.87      0.88       993

